# Collage

We'll create a simple collage from images: 

* Select a random subset of images
* For each image
    * Get a square from the center of the image
    * Resize the square to 256x256
* Put the images together in a tile of size `tile_h` x `tile_w`

In [ ]:
tile_h = 10
tile_w = 25

total = tile_h * tile_w

In [ ]:
from glob import glob
from random import shuffle
from math import ceil, floor

from PIL import Image

Let's define a few functions:

* `center_crop` for making an image square
* `resize` for resizing the image to 256x256
* `crop_resize` for combining these two functions
* `concat_horizontal` and `concat_vertical` for concatenating multiple images horizontally and vertically 

In [ ]:
def center_crop(img):
    h = img.height
    w = img.width
    d = min(w, h)

    if h == w:
        return img

    if h > w:
        left = 0
        right = w
        top = floor((h - d) / 2)
        bottom = h - top - 1
    else:
        left = floor((w - d) / 2)
        right = w - left - 1
        top = 0
        bottom = h

    return img.crop((left, top, right, bottom))


def resize(img, d=256):
    h = img.height
    w = img.width

    if h > w:
        new_h = d
        new_w = floor(h * d / w)
    else:
        new_h = floor(w * d / h)
        new_w = d

    return img.resize((new_h, new_w),resample=Image.BICUBIC)


def crop_resize(img):
    return center_crop(resize(img))

In [ ]:
def concat_horizontal(images):
    w = sum(im.width for im in images)
    h = max(im.height for im in images)

    result = Image.new('RGB', (w, h))

    start_w = 0
    for im in images:
        result.paste(im, (start_w, 0))
        start_w = start_w + im.width

    return result

def concat_vertical(images):
    w = max(im.width for im in images)
    h = sum(im.height for im in images)

    result = Image.new('RGB', (w, h))

    start_h = 0
    for im in images:
        result.paste(im, (0, start_h))
        start_h = start_h + im.height
    
    return result

Now, let's create a collage:

* select a subset of images
* open each wit PIL
* crop and resize
* concat vertically and then horizontally
* save the result

In [ ]:
images = glob('../input/clothing-dataset-full/images_compressed/*')
shuffle(images)

In [ ]:
images_pil = [Image.open(im) for im in images[:total]]

In [ ]:
cropped = []

for img in images_pil:
    c = crop_resize(img)
    cropped.append(c)

In [ ]:
lines = []

for i in range(0, total, tile_w):
    sublist = cropped[i:i + tile_w]
    line = concat_horizontal(sublist)
    lines.append(line)


result = concat_vertical(lines)

In [ ]:
result.save('collage.jpg')

In [ ]:
Image.open('../working/collage.jpg')